# Workflow API Tutorial: Keras MNIST

This tutorial demonstrates how to train a model using TensorFlow Keras on the MNIST dataset, leveraging the new OpenFL Workflow Interface. The Workflow Interface provides a novel way to compose federated learning experiments with OpenFL, enabling researchers to handle non-IID data and perform federated averaging with optimizations like FedProx. Through this tutorial, you will learn how to set up the federated learning environment, define the flow, and execute the training process across multiple collaborators.


# What is it?

The workflow interface is a new way of composing federated learning expermients with OpenFL. It was born through conversations with researchers and existing users who had novel use cases that didn't quite fit the standard horizontal federated learning paradigm. 

# Getting Started

First we start by installing the necessary dependencies for the workflow interface. Note that this tuorial uses Keras 3, make sure you use python 3.9 or higher.

In [ ]:
%pip install tensorflow
%pip install openfl
%pip install -r workflow_interface_requirements.txt

Now, we import the relevant modules and do some basic initializations  

In [ ]:
import numpy as np
import tensorflow as tf

tf.data.experimental.enable_debug_mode()
tf.random.set_seed(0)
np.random.seed(0)

The FedAvg method is used in our federated learning architecture to aggregate model weights from multiple collaborators. In this approach, each collaborator trains a local model on their local data and then sends the model weights to a central aggregator. The aggregator runs the FedAvgWeights method to calculate the aggregated model weights by averaging the weights from all collaborators. 

In [ ]:
import numpy as np

def FedAvg(models):
    new_model = models[0]
    state_dicts = [model.weights for model in models]
    state_dict = new_model.weights
    for idx, _ in enumerate(models[1].weights):
        state_dict[idx] = np.sum(np.array([state[idx]
                                 for state in state_dicts], dtype=object), axis=0) / len(models)
    new_model.set_weights(state_dict)
    return new_model

Now we come to the flow definition. The OpenFL Workflow Interface adopts the conventions set by Metaflow, that every workflow begins with `start` and concludes with the `end` task. The aggregator begins with an optionally passed in model and optimizer. The aggregator begins the flow with the `start` task, where the list of collaborators is extracted from the runtime (`self.collaborators = self.runtime.collaborators`) and is then used as the list of participants to run the task listed in `self.next`, `aggregated_model_validation`. The model, optimizer, and anything that is not explicitly excluded from the next function will be passed from the `start` function on the aggregator to the `aggregated_model_validation` task on the collaborator. Where the tasks run is determined by the placement decorator that precedes each task definition (`@aggregator` or `@collaborator`). Once each of the collaborators (defined in the runtime) complete the `aggregated_model_validation` task, they pass their current state onto the `train` task, from `train` to `local_model_validation`, and then finally to `join` at the aggregator. It is in `join` that an average is taken of the model weights, and the next round can begin.

In [ ]:
from openfl.experimental.workflow.interface import FLSpec
from openfl.experimental.workflow.placement import aggregator, collaborator
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def inference(model, test_loader, batch_size):
    x_test, y_test = test_loader
    loss, accuracy = model.evaluate(
        x_test,
        y_test,
        batch_size=batch_size,
        verbose=1
    )
    accuracy_percentage = accuracy * 100
    print(f"Test set: Avg. loss: {loss}, Accuracy: {accuracy_percentage:.2f}%")
    return accuracy

class KerasMNISTFlow(FLSpec):
    def __init__(self, model, rounds=3, **kwargs):
        super().__init__(**kwargs)
        self.model = model
        self.n_rounds = rounds
        self.current_round = 1

    @aggregator
    def start(self):
        self.collaborators = self.runtime.collaborators
        self.next(self.aggregated_model_validation, foreach='collaborators')

    @collaborator
    def aggregated_model_validation(self):
        print(f'Performing aggregated model validation for collaborator {self.input}')
        self.agg_validation_score = inference(self.model, self.test_loader, self.batch_size)
        print(f'{self.input} value of {self.agg_validation_score}')
        self.next(self.train)

    @collaborator
    def train(self):
        x_train, y_train = self.train_loader
        print(f'x_train shape: {x_train.shape}, y_train shape: {y_train.shape}')
        history = self.model.fit(
            x_train, y_train,
            batch_size=self.batch_size,
            epochs=1,
            verbose=1,
        )
        self.loss = history.history["loss"][0]
        self.next(self.local_model_validation)

    @collaborator
    def local_model_validation(self):
        self.local_validation_score = inference(self.model, self.test_loader, self.batch_size)
        print(
            f'Doing local model validation for collaborator {self.input}: {self.local_validation_score}')
        self.next(self.join)

    @aggregator
    def join(self, inputs):
        self.average_loss = sum(input.loss for input in inputs) / len(inputs)
        self.aggregated_model_accuracy = sum(
            input.agg_validation_score for input in inputs) / len(inputs)
        self.local_model_accuracy = sum(
            input.local_validation_score for input in inputs) / len(inputs)
        print(f'Average aggregated model validation values = {self.aggregated_model_accuracy}')
        print(f'Average training loss = {self.average_loss}')
        print(f'Average local model validation values = {self.local_model_accuracy}')
        print("Taking FedAvg of models of all collaborators")
        self.model = FedAvg([input.model for input in inputs])

        self.next(self.internal_loop)

    @aggregator
    def internal_loop(self):
        if self.current_round == self.n_rounds:
            self.next(self.end)
        else:
            self.current_round += 1
            self.next(self.aggregated_model_validation, foreach='collaborators')

    @aggregator
    def end(self):
        print(f'This is the end of the flow')

We simulate the data sharing in a lognormal distribution. We split the train data set to a train data set and local validation data set such that we can test the aggregated model on a new fresh data

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np

VALID_PERCENT = 0.3

nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255.0
X_test /= 255.0


split_on = int((1 - VALID_PERCENT) * len(X_train))

train_images = X_train[0:split_on,:,:]
train_labels = to_categorical(y_train)[0:split_on,:]

test_images = X_train[split_on:,:,:]
test_labels = to_categorical(y_train)[split_on:,:]

valid_images =  X_test
valid_labels = to_categorical(y_test)

print("Training matrix shape", train_images.shape)
print("Validation matrix shape", valid_images.shape)
print("Test matrix shape", test_images.shape)

class LogNormalNumPyDataSplitter:
    def __init__(self, num_clients, num_classes=10, mu=0, sigma=1):
        self.num_clients = num_clients
        self.num_classes = num_classes
        self.mu = mu
        self.sigma = sigma

    def split(self, images, labels):
        # Convert labels to one-hot encoding if they are not already
        if len(labels.shape) == 1 or labels.shape[1] == 1:
            labels = tf.keras.utils.to_categorical(labels, num_classes=self.num_classes)
        
        # Get the class indices for each label
        labels = np.argmax(labels, axis=1)
        
        # Initialize the indices for each client
        client_indices = [[] for _ in range(self.num_clients)]
        
        # Calculate the lognormal proportions
        props = np.random.lognormal(self.mu, self.sigma, (self.num_classes, self.num_clients))
        props = props / np.sum(props, axis=1, keepdims=True)
        
        # Distribute the indices based on the lognormal proportions
        for class_idx in range(self.num_classes):
            class_indices = np.where(labels == class_idx)[0]
            np.random.shuffle(class_indices)
            
            # Calculate the split points based on the proportions
            split_points = (np.cumsum(props[class_idx])[:-1] * len(class_indices)).astype(int)
            
            split_indices = np.split(class_indices, split_points)
            
            for client_idx, indices in enumerate(split_indices):
                client_indices[client_idx].extend(indices)
        
        return client_indices


Now we initiate federated components.
You'll notice in the `FederatedFlow` definition above that there were certain attributes that the flow was not initialized with, namely the `train_loader` and `test_loader` for each of the collaborators. These are **private_attributes** of the particular participant and (as the name suggests) are accessible ONLY to the particular participant's through its task. Additionally these private attributes are always filtered out of the current state when transferring from collaborator to aggregator, and vice versa.
 
Users can directly specify a collaborator's private attributes via `collaborator.private_attributes` which is a dictionary where key is name of the attribute and value is the object that is made accessible to collaborator. In this example, we segment shards of the MNIST dataset for four collaborators: `Portland`, `Seattle`, `Chandler`  and `Bangalore`. Each shard / slice of the dataset is assigned to collaborator's private_attribute.
 
Note that the private attributes are flexible, and user can choose to pass in a completely different type of object to any of the collaborators or aggregator (with an arbitrary name).

In [ ]:
from openfl.experimental.workflow.interface import Collaborator, Aggregator
from openfl.experimental.workflow.runtime import LocalRuntime

def callable_to_initialize_collaborator_private_attributes(n_collaborators, index, train_dataset, test_dataset, batch_size):
    train_splitter = LogNormalNumPyDataSplitter(num_clients=n_collaborators,num_classes=10, mu=0, sigma=1)
    test_splitter = LogNormalNumPyDataSplitter(num_clients=n_collaborators,num_classes=10, mu=0, sigma=1)


    X_train, y_train = train_dataset
    X_test, y_test = test_dataset
    train_idx = train_splitter.split(X_train, y_train)
    valid_idx = test_splitter.split(X_test, y_test)

    train_dataset = X_train[train_idx[index]], y_train[train_idx[index]]
    test_dataset = X_test[valid_idx[index]], y_test[valid_idx[index]]

    return {
        "train_loader": train_dataset, "test_loader": test_dataset,
        "batch_size": batch_size
    }
    
# # Setup participants
aggregator = Aggregator()
# aggregator.private_attributes = {}
collaborators = []
collaborator_names = ['Portland', 'Seattle', 'Chandler','Bangalore']
for idx, collaborator_name in enumerate(collaborator_names):
    collaborators.append(
        Collaborator(
            name=collaborator_name, num_cpus=1, num_gpus=0,
            private_attributes_callable=callable_to_initialize_collaborator_private_attributes,
            n_collaborators=len(collaborator_names), index=idx, train_dataset=(train_images, train_labels),
            test_dataset=(test_images, test_labels), batch_size=32
        )
    )
local_runtime = LocalRuntime(aggregator=aggregator, collaborators=collaborators, backend='single_process')
print(f'Local runtime collaborators = {local_runtime.collaborators}')


Now that we have all the building blocks in place, we define the model and run a validation session on the non trained model.

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from openfl.utilities.optimizers.keras import FedProxOptimizer
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    MaxPool2D(),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.2),
    Dense(512, activation="relu"),
    Dropout(0.2),
    Dense(nb_classes, activation="softmax"),
])

model.compile(optimizer=FedProxOptimizer(mu=0.01, learning_rate=0.01), loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])
model.summary()

flflow = KerasMNISTFlow(model, rounds=3, checkpoint=True)
flflow.runtime = local_runtime
inference(flflow.model, (test_images, test_labels), 32)

Now, let's train the model using the federated flow as we defined above.

In [ ]:
flflow.run()

Now, let's train the model using the federated setup

Now, after the training, let's evaluate the aggregated model accuracy again

In [ ]:
inference(flflow.model, (test_images, test_labels), 32)

# Congratulations!
Now that you've completed your first workflow interface quickstart notebook, see some of the more advanced things you can do in our other tutorials, including:

- Using the LocalRuntime Ray Backend for dedicated GPU access
- Vertical Federated Learning
- Model Watermarking
- Differential Privacy
- And More!